## Description

Наша виртуальная машина содержит следующие части:
* **Стек** - Варьируемое количество строк, задаваемое в начале программы на ассемблере/просто строками в бинарном коде
* **Регистры** - часть стека, первые 16 ячеек, именные (reg1, reg2, ...)
* **Константы** - задаваемые программистом текстовые и целочисленные величины
* **Программа** - собственно исполняемая программа

В байт-коде первая строка - **ip**, затем идет **n** - размер стека, после чего идет n ячеек фиксированного стека, после этого константы и сама программа. Каждая команда состоит из собственно кода команды и трех параметров, двух однобайтных и одного с варьируемой длиной, в зависимости от команды.
В ассемблере есть четыре ключевых слова (регистр важен) - *STRING_VARS*, *FUNCTIONS*, *START*, *MEMORY*. Программе не важно, в каком порядке они идут и есть ли они вообще, но важно, чтобы каждый блок встречался не более одного раза. В коде программы можно оставлять лейблы, не являющиеся ключевыми словами или названиями операций, по ним производится навигация с помощью команд типа JMP. 

Перечень команд (*по умолчанию синтаксис OPERATION reg1 reg2 c, где reg1 и reg2 - произвольные регистры стека, c - целое число или регистр в зависимости от команды*):
* *ADD* - сложение, принимает на вход три регистра, производит операцию reg3 = reg1 + reg2;
* *SUB* - вычитание, принимает на вход три регистра, производит операцию reg3 = reg1 - reg2;
* *MUL* - умножение, принимает на вход три регистра, производит операцию reg3 = reg1 * reg2;
* *PRINT_STR* - вывод строки из блока констант. Принимает на вход имя переменной в Assemblere/индекс в памяти в байт-коде;
* *JMP* - безусловный переход по индексу/лейблу:
* *JMP_ZERO* - переход по индексу/лейблу, если в reg1 лежит 0;
* *PUSH* - положить на стек значение из reg1;
* *POP* - вытащить со стека верхнее значение в reg1;
* *READ* - считывание числа из ввода, кладет в reg1;
* *PRINT_INT* - вывод числа из reg1;
* *MOVE* - положить в reg1 значение c (c - 4 байта);
* *COPY* - скопировать значение из reg2 в reg1;
* *CALL* - вызов функции (см. соглашение о вызовах);
* *RET* - возврат из функции;
* *STOP* - остановка исполнения, конец программы.


Для запуска программы необходимо передать в класс **VirtualMachine** имя бинарного файла и выполнить его метод *Run*. Для исполнения ассемблерного файла необходимо передать в класс **Assembler** имя файла с ассемблерной программой и название бинарного файла (по умолчанию *program.bin*), после чего будет сгенерирован бинарный файл, который можно передать в виртуальную машину.

**Стек.** 
Управление стеком осуществляется с помощью push и pop и остается полностью на совести программиста. Если он положил слишком много или вытащил больше, чем положил, начнут затираться ячейки программы и прочая важная информация, то есть задача не выстрелить себе в ногу.

**Соглашение о вызовах.**
При вызове ip кладется на стек, при возврате происходит переход по верхнему значению на стеке. Для передачи параметров можно использовать регистры, главное, что контроль за памятью полностью ложится на плечи программиста. Это означает, что если он хочет сохранить перед вызовом функции регистры, он должен их положить на стек. Возвращаемое значение может лежать в любых регистрах, главное, чтобы программист знал, где. Негласное соглашение - возвращаемое значение надо класть в reg1. Так же программист может пользоваться стеком, главное не забывать всё с него удалять!

## Operations and other sintax

In [65]:
operations = {"ADD": 0x01, 
               "SUB": 0x02, 
               "MUL": 0x03,
               "PRINT_STR": 0x04,
               "JMP": 0x05,
               "JMP_ZERO": 0x06, 
               "PUSH": 0x07,
               "READ": 0x08,
               "PRINT_INT": 0x09,
               "MOVE": 0x0A,
               "CALL": 0x0B,
               "RET": 0xEE,
               "POP": 0x0C,
               "COPY": 0x0D,
               "STOP": 0xFF
              }
operations_codes = {val: key for key, val in operations.items()}

## Utilities

In [66]:
 def convert_int_to_hex(num):
    num = str(hex(num))[2:]
    num = '0' * (8 - len(num)) + num
    num = [num[i:i + 2] for i in range(0, 8, 2)]
    return num

In [67]:
def convert_string_to_hex(string):
    res = ''
    for symbol in string:
        hex_symb = str(hex(ord(symbol)))[2:]
        hex_symb = '0' * (4 - len(hex_symb)) + hex_symb
        res += hex_symb[:2] + ' ' + hex_symb[2:] + ' '
    return res

## Virtual Machine

In [78]:
from copy import copy

class VirtualMachine():
    def __init__(self, filename):
        self.Load(filename)
        self.ip = int(''.join(self.program[0]), 16)
        self.stack_size = int(''.join(self.program[1]), 16)
        self.sp = 18 # stack pointer
        # All the operations has the same syntax: 
        # *code/name of op - 1 byte* *reg1 - 1 byte* *reg2 - 1 byte* *optional value, depends on operation, n bytes*
        
        self.operations_functions = {0x01: self.add, # 01 a b c -> c = a + b
                                     0x02: self.sub, # 02 a b c -> c = a - b
                                     0x03: self.mul, # 03 a b c -> c = a + b
                                     0x04: self.print_str, # 04 a b c -> c>>stdout (c -  4 bytes, adress of str)
                                     0x05: self.jmp, # 05 a b c -> ip = c (c - 4 bytes)
                                     0x06: self.jmp_zero, # 06 a b c -> ip = (a == 0) ? c : ip (c - 4 bytes)
                                     0x07: self.push, # 07 a b c -> a -> stack 
                                     0x08: self.read, # 08 a b c -> a = int(input())
                                     0x09: self.print_int, # 09 a b c -> a>>stdout (a - int)
                                     0x0A: self.move, # 0A a b c -> a = c (c - int)
                                     0x0B: self.call, # 0B a b c -> ip = c (c - int)
                                     0xEE: self.ret,
                                     0x0C: self.pop, # 0C a b c -> stack(top) -> a 
                                     0x0D: self.copy, # 0D a b c -> b = a
                                     0xFF: self.stop # FF a b c -> ip = None
                                     }
        
    def Load(self, filename):
        self.program = []
        with open(filename) as f:
            content = f.readlines()
        for line in content:
            self.program.append(line.split())
    
    def Run(self):
        while self.ip:
            self.operations_functions[int(self.program[self.ip][0], 16)]()
    
    
    def push(self):
        reg1 = int(self.program[self.ip][1], 16)
        self.program[self.sp] = copy(self.program[reg1])
        self.sp += 1
        self.ip += 1
        
    def pop(self):
        reg1 = int(self.program[self.ip][1], 16)
        self.sp -= 1
        self.program[reg1] = copy(self.program[self.sp])
        self.ip += 1
        
    def copy(self):
        reg1 = int(self.program[self.ip][1], 16)
        reg2 = int(self.program[self.ip][2], 16)
        self.program[reg2] = copy(self.program[reg1])
        self.ip += 1
        
    def move(self):
        reg1 = int(self.program[self.ip][1], 16)
        num1 = self.program[self.ip][3:]
        self.program[reg1] = copy(num1)
        self.ip += 1
    
    def add(self): 
        reg1 = int(self.program[self.ip][1], 16)
        reg2 = int(self.program[self.ip][2], 16)
        reg3 = int(self.program[self.ip][3], 16)
        
        num1 = int(''.join(self.program[reg1]), 16)
        num2 = int(''.join(self.program[reg2]), 16)        
        res = convert_int_to_hex(num1 + num2)  
        
        self.program[reg3] = copy(res)
        self.ip += 1
        
    def sub(self): 
        reg1 = int(self.program[self.ip][1], 16)
        reg2 = int(self.program[self.ip][2], 16)
        reg3 = int(self.program[self.ip][3], 16)
        
        num1 = int(''.join(self.program[reg1]), 16)
        num2 = int(''.join(self.program[reg2]), 16)        
        res = convert_int_to_hex(num1 - num2)  
        
        self.program[reg3] = copy(res)
        self.ip += 1
        
    def mul(self): 
        reg1 = int(self.program[self.ip][1], 16)
        reg2 = int(self.program[self.ip][2], 16)
        reg3 = int(self.program[self.ip][3], 16)
        
        num1 = int(''.join(self.program[reg1]), 16)
        num2 = int(''.join(self.program[reg2]), 16)        
        res = convert_int_to_hex(num1 * num2)  
        
        self.program[reg3] = copy(res)
        self.ip += 1
    
    def print_str(self):
        reg1 = int(''.join(self.program[self.ip][3:]), 16)
        
        text_to_print = [chr(int(''.join(self.program[reg1][i:i + 2]), 16)) for i in range(0, len(self.program[reg1]), 2)]
        text_to_print = ''.join(text_to_print)
        print(text_to_print)
        self.ip += 1
    
    def print_int(self):
        reg1 = int(self.program[self.ip][1], 16)
        num1 = int(''.join(self.program[reg1]), 16)
        print(num1)
        self.ip += 1
        
    def read(self):
        reg1 = int(self.program[self.ip][1], 16)
        num1 = int(input())
        self.program[reg1] = copy(convert_int_to_hex(num1))
        self.ip += 1
    
    def jmp(self):
        self.ip = int(''.join(self.program[self.ip][3:]), 16)
        
    def jmp_zero(self):
        reg1 = int(self.program[self.ip][1], 16)
        num1 = int(''.join(self.program[reg1]), 16)
        if num1 == 0:
            self.ip = int(''.join(self.program[self.ip][3:]), 16)
        else:
            self.ip += 1
        
    def call(self):
        self.program[self.sp] = convert_int_to_hex(self.ip)
        self.sp += 1
        self.ip = int(''.join(self.program[self.ip][3:]), 16)
        
    def ret(self):
        self.sp -= 1
        self.ip = int(''.join(self.program[self.sp]), 16)
        self.ip += 1
        
    
    def stop(self):
        print()
        print(".............")
        print("Program finished executing")
        self.ip = None
        

## Assembler

In [82]:
class Assembler():
    def __init__(self, filename, bin_file = "program.bin"):
        
        self.key_words = ["STRING_VARS", "START", "MEMORY", "FUNCTIONS"]
        self.parse_functions = {'ADD': self.add, # ADD a b c -> c = a + b (a, b, c - regs)
                                 'SUB': self.sub, # SUB a b c -> c = a - b (a, b, c - regs)
                                 'MUL': self.mul, # MUL a b c -> c = a * b (a, b, c - regs)
                                 'PRINT_STR': self.print_str, # PRINT_STR a -> a>>stdout (only from variables) (a - var_name)
                                 'PRINT_INT': self.print_int, # PRINT_INT a -> a>>stdout (only from registers) (a - reg)
                                 'READ': self.read, # READ a -> a = int(input()) (a - reg)
                                 'JMP': self.jmp, # JMP a -> ip = a (a - label)
                                 'JMP_ZERO': self.jmp_zero, # JMP_ZERO a b -> ip = (a == 0) ? b : ip + 1 (a - reg, b - label)
                                 'PUSH': self.push, # PUSH a -> a >> stack (a - reg)
                                 'POP': self.pop, # POP a -> stack(top) >> a
                                 'MOVE': self.move, # MOVE a c -> a = c (c - int)
                                 'COPY': self.copy, # COPY a b -> b = a (a, b - regs)
                                 'CALL': self.call,
                                 'RET': self.ret,
                                 'STOP': self.stop # STOP -> ip = None
                                 }
        self.Load(filename)
        self.CreateBinFile(bin_file)
        
    def Load(self, filename):
        
        with open(filename, encoding="utf-8") as f:
            self.content = f.readlines()
        self.content = [string.split('\n')[0] for string in self.content]
        
        self.GetMemory()
        self.program = [' '.join(convert_int_to_hex(0)) ] * self.memory
        
        self.vars = {}
        self.vars_count = 0
        self.GetStringVars()
        
        self.functions = {}
        self.GetFunctions()
        
        ip = len(self.program) + 1
        self.program = [' '.join(convert_int_to_hex(ip))] + self.program
        
        self.GetProgram()
        
    def CreateBinFile(self, bin_file):
        with open(bin_file, 'w', encoding="utf-8") as f:
            f.write('\n'.join(self.program))
        
    def GetMemory(self):
        if "MEMORY" in self.content:
            self.memory = int(self.content[self.content.index("MEMORY") + 1])
        else:
            self.memory = 16
        self.regs = {'reg' + str(i): i for i in range(self.memory)}
    
    def GetStringVars(self):
        if 'STRING_VARS' in self.content:
            line = self.content.index('STRING_VARS') + 1
            while line != len(self.content) and self.content[line] not in self.key_words:
                var = self.content[line].split("\"")
                self.program.append(convert_string_to_hex(var[1]))
                self.vars[var[0][:-1]] = self.vars_count + 1
                self.vars_count += 1
                line += 1
        
    def GetFunctions(self):
        if 'FUNCTIONS' in self.content:
            line = self.content.index('FUNCTIONS') + 1
            
            while line < len(self.content) and self.content[line] not in self.key_words:
                command = self.content[line].split()
                if command[0] == "DEF":
                    self.functions[command[1]] = len(self.program) + 1
                    self.labels = {}
                    self.jumps = {}
                    line += 1
                    while(self.content[line] != "RET"):
                        command = self.content[line].split()
                        if command[0] in self.parse_functions:
                            self.parse_functions[command[0]](command)
                        else:
                            self.labels[command[0]] = len(self.program) + 1
                        line += 1
                    self.ret()
                    for line, label in self.jumps.items():
                        self.program[line - 1] += ' '.join(convert_int_to_hex(self.labels[label]))
                else:
                    line += 1  
        
    def GetProgram(self):
        line = self.content.index('START') + 1
        self.labels = {}
        self.jumps = {}
        while line != len(self.content) and self.content[line] not in self.key_words:
            command = self.content[line].split()
            if command[0] in self.parse_functions:
                    self.parse_functions[command[0]](command)
            else:
                self.labels[command[0]] = len(self.program)
            line += 1
        for line, label in self.jumps.items():
            self.program[line - 1] += ' '.join(convert_int_to_hex(self.labels[label]))    
            
    def add(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
            ' '.join([convert_int_to_hex(self.regs[command[i]])[3] for i in range(1, 4)])
        self.program.append(res)
    
    def sub(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
            ' '.join([convert_int_to_hex(self.regs[command[i]])[3] for i in range(1, 4)])
        self.program.append(res)
    
    def mul(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
            ' '.join([convert_int_to_hex(self.regs[command[i]])[3] for i in range(1, 4)])
        self.program.append(res)
    
    def print_int(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
                convert_int_to_hex(self.regs[command[1]])[3] + ' 00 00'
        self.program.append(res)
    
    def print_str(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' 00 00 ' + \
                ' '.join(convert_int_to_hex(self.vars[command[1]] + self.memory))
        self.program.append(res)
        
    def read(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
                convert_int_to_hex(self.regs[command[1]])[3] + ' 00 00'
        self.program.append(res)
        
    def jmp(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' 00 00 '
        self.program.append(res)
        self.jumps[len(self.program)] = command[1]
    
    def jmp_zero(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
                convert_int_to_hex(self.regs[command[1]])[3] + ' 00 '
        self.program.append(res)
        self.jumps[len(self.program)] = command[2]
    
    def push(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
                convert_int_to_hex(self.regs[command[1]])[3] + ' 00 00'
        self.program.append(res)
        
    def pop(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
                convert_int_to_hex(self.regs[command[1]])[3] + ' 00 00'
        self.program.append(res)
        
    def move(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
            convert_int_to_hex(self.regs[command[1]])[3]+ ' 00 ' + ' '.join(convert_int_to_hex(int(command[2])))
        self.program.append(res)
    
    def copy(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' ' + \
            ' '.join([convert_int_to_hex(self.regs[command[i]])[3] for i in range(1, 3)]) + ' 00'
        self.program.append(res)
    
    def call(self, command):
        res = convert_int_to_hex(operations[command[0]])[3] + ' 00 00 ' + \
                ' '.join(convert_int_to_hex(self.functions[command[1]]))
        self.program.append(res)
        
    def ret(self):
        self.program.append('ee 00 00 00')
    
    def stop(self, command):
        self.program.append('ff 00 00 00')

## Subtasks

#### 1. Hello world

In [83]:
vm = VirtualMachine('hello_world.bin')
vm.Run()

Привет, Мир!

.............
Program finished executing


#### 2. Fibonacci

In [105]:
asm = Assembler('fibonacci.asm')
vm = VirtualMachine('program.bin')
vm.Run()

Привет, эта программа вычисляет n-тое число Фибоначчи!
Пожалуйста, введите n:
15
Вот ваше число:
987

.............
Program finished executing


#### 3. Fibonacci (recursion)

In [104]:
asm = Assembler('fibonacci(recursion).asm')
vm = VirtualMachine('program.bin')
vm.Run()

Привет, эта программа рекурсивно вычисляет n-тое число Фибоначчи!
Пожалуйста, введите n:
15
Вот ваше число:
987

.............
Program finished executing
